In [1]:
import pandas as pd
from pathlib import Path
import qgrid
import sys

sys.path.insert(0, '..')
import Scoreboard19 as S19

In [2]:
S19.figures_dir = Path('../Figures').resolve()
S19.data_dir = Path('../Data').resolve()

In [3]:
regenerate = False
if regenerate:
    case_forecasts = S19.getscoreboard(US_cases, 'Case', '../Data/ScoreboardCases.pkl')
    case_forecasts.to_pickle('../Data/ModelsANDForecasts.pkl')
    death_forecasts = S19.getscoreboard(US_deaths,'Death','../Data/ScoreboardDeaths.pkl')

In [4]:
#US_cases = S19.readobservedcases()
#US_deaths = S19.readobserveddeaths()

In [5]:
#FirstForecasts = pd.read_pickle(S19.data_dir / 'ModelsANDForecasts.pkl')
#S19.numberofteamsincovidhub(FirstForecasts, S19.figures_dir)

In [6]:
scoreboard_cases = pd.read_pickle(S19.data_dir / 'ScoreboardCases.pkl')
scoreboard_deaths = pd.read_pickle(S19.data_dir / 'ScoreboardDeaths.pkl')

In [7]:
#%%capture cap --no-stderr
#themodels = ScoreboardCases['model'].unique()
#for i in range(0,len(themodels)):
#    print(themodels[i])

In [8]:
#Writing out all the models in the scoreboard into a text file
#with open('../INCCASE_Models.txt', 'w') as f:
#    f.write(cap.stdout)

In [9]:
scoreboard_cases = S19.fix_scoreboard(scoreboard_cases, kind='Case', quiet=True, plot=False)

In [10]:
scoreboard_deaths = S19.fix_scoreboard(scoreboard_deaths, kind='Death', quiet=True, plot=False)

In [11]:
#S19.plotallscoresdist(ScoreboardCases, 'Case');

In [12]:
#S19.plotlongitudinalALL(US_cases, ScoreboardCases, 'Cases', 1)

In [13]:
#S19.plotlongitudinalALL(US_cases, ScoreboardCases, 'Cases', 4)

In [14]:
#S19.plotlongitudinal(US_cases, ScoreboardCases, 'Cases', 1, ['BPagano:RtDriven'])

In [15]:
#S19.plotlongitudinal(US_cases, ScoreboardCases, 'Cases', 1, ['COVIDhub:ensemble'])

In [16]:
#S19.plotlongitudinal(US_cases, ScoreboardCases, 'Cases', 1, ['COVIDhub:baseline'])

In [17]:
#S19.plotlongitudinal(US_cases, ScoreboardCases, 'Cases', 1, ['CEID:Walk'])

In [18]:
#S19.plotUSCumDeaths(US_deaths)

In [19]:
#%%capture cap --no-stderr
#themodels = ScoreboardDeaths['model'].unique()
#for i in range(0,len(themodels)):
#    print(themodels[i])

In [20]:
#Writing out all the models in the scoreboard into a text file
#with open('../CUMDEATH_Models.txt', 'w') as f:
#    f.write(cap.stdout)

In [21]:
#S19.plotallscoresdist(ScoreboardDeaths, 'Death')

In [22]:
#S19.plotlongitudinalALL(US_deaths, ScoreboardDeaths, 'Deaths', 1)

In [23]:
#S19.plotlongitudinalALL(US_deaths, ScoreboardDeaths, 'Deaths', 4)

In [24]:
#S19.plotlongitudinal(US_deaths, ScoreboardDeaths, 'Deaths', 1, ['BPagano:RtDriven'])

In [25]:
#S19.plotlongitudinal(US_deaths, ScoreboardDeaths, 'Deaths', 1, ['COVIDhub:baseline'])

In [26]:
#S19.plotlongitudinal(US_deaths, ScoreboardDeaths, 'Deaths', 4, ['COVIDhub:baseline'])

In [27]:
#Number of Unique Case Forecasting Models in the data set 
#len(ScoreboardCases['model'].unique())

In [28]:
#Number of Unique Cumulative Death Forecasting Models in the data set 
#len(ScoreboardDeaths['model'].unique())

In [11]:
weight_date = '2020-08-01'
n_weeks = 4
#scoreboard_cases = S19.getweightedmodelalldates(scoreboard_cases, weight_date, 'Cases', n_weeks, 'weighted')
#scoreboard_cases = S19.getweightedmodelalldates(scoreboard_cases, weight_date, 'Cases', n_weeks, 'unweighted')
#scoreboard_deaths = S19.getweightedmodelalldates(scoreboard_deaths, weight_date, 'Deaths', n_weeks, 'weighted')
#scoreboard_deaths = S19.getweightedmodelalldates(scoreboard_deaths, weight_date, 'Deaths', n_weeks, 'unweighted')

In [30]:
#S19.plotlongitudinal(US_deaths, ScoreboardDeaths, 'Deaths', 4, ['COVIDhub:ensemble','FDANIH:Sunweight'])

In [31]:
#S19.plotlongitudinalUNWEIGHTED(US_deaths, ScoreboardDeaths, 'Deaths', 7)

In [32]:
#S19.plotlongitudinal(US_cases, ScoreboardCases, 'Cases', 4, ['COVIDhub:ensemble','FDANIH:Sunweight'])

In [33]:
#S19.plotlongitudinalUNWEIGHTED(US_cases, ScoreboardCases, 'Cases', 7)

In [34]:
#S19.plotlongitudinal(US_deaths, ScoreboardDeaths, 'Deaths', 4, ['COVIDhub:ensemble','FDANIH:Sweight'])

In [35]:
#S19.plotlongitudinal(US_cases, ScoreboardCases, 'Cases', 4, ['COVIDhub:ensemble','FDANIH:Sweight'])

In [36]:
#S19.plotgroupsFD(ScoreboardDeaths, modeltypesDeaths, 1, 'Death')

In [37]:
#S19.plotgroupsFD(ScoreboardCases, modeltypesCases, 1, 'Case')

In [38]:
#S19.plotgroupsmodelweek(ScoreboardCases, modeltypesCases, 4, 'Case')

In [39]:
#S19.plotgroupsmodelweek(ScoreboardDeaths, modeltypesDeaths, 4, 'Death')

In [40]:
#S19.plotgroupsTD(ScoreboardDeaths, modeltypesDeaths, 'Death')

In [41]:
#S19.plotgroupsTD(ScoreboardCases, modeltypesCases, 'Case')

In [42]:
#S19.getleaderboard(scoreboard_cases, 1)

In [43]:
#S19.getleaderboard(scoreboard_cases, 4)

In [44]:
#S19.getleaderboard(scoreboard_deaths, 1)

In [45]:
#S19.getleaderboard(scoreboard_deaths, 4)

In [12]:
import ipywidgets as ipyw
from ipywidgets import HBox, VBox, Text, HTML, Label, AppLayout

In [13]:
def get_qgrid(scoreboard, weeks_ahead):
    df = S19.getleaderboard(scoreboard, weeks_ahead, quiet=True)
    df.columns = [x.title().replace('Of ', '').replace('Rankings', 'Rank').replace('Average', 'Mean').replace('Past ', '').replace('Scores', 'Score')
                  for x in df.columns]
    df.iloc[:, 1] = df.iloc[:, 1].round(2)
    df.iloc[:, 2] = df.iloc[:, 2].round(1)
    df['Team'] = df['Model'].apply(lambda x: x.split(':')[0])
    df['Model'] = df['Model'].apply(lambda x: x.split(':')[1])
    df = df.set_index('Team')
    #df.index.name = "Team:Model"
    return qgrid.show_grid(df, grid_options={'width': 400, 'forceFitColumns': False, 'defaultColumnWidth': 110})

In [19]:
S19.getleaderboard(scoreboard_cases,4)

Leaderboard for 4-week-ahead weekly incidental case forecasts as of 2020-12-12


,model,median of past scores,average of past rankings
0,CU:scenario_low,-10.322189,5.625000
1,CU:scenario_mid,-10.778736,5.750000
2,CU:select,-10.790454,6.250000
3,COVIDhub:baseline,-10.990190,5.000000
4,CU:nochange,-11.128248,6.687500
5,CU:scenario_high,-11.250869,7.250000
6,DDS:NBDS,-11.712890,7.117647
7,UVA:Ensemble,-11.814411,12.000000
8,UCLA:SuEIR,-11.926025,7.250000
9,LNQ:ens1,-11.965882,6.473684


In [14]:
weeks_ahead = 4
q_cases = get_qgrid(scoreboard_cases, weeks_ahead)
q_deaths = get_qgrid(scoreboard_cases, weeks_ahead)
q_cases.layout = ipyw.Layout(padding='10px', width='458px', align_items='center')
q_deaths.layout = ipyw.Layout(padding='10px', width='458px', align_items='center')

In [15]:
%%html
<style>
.widget-label {
    text-align: center;
    font-size: 150%
}
.q-grid-container {
    text-align: center;
}
</style>

In [16]:
centered = ipyw.Layout(align_items='center', align_self='center')

In [17]:
title = Label('COVIDHub Model Dashboard')
VBox([title,
      HBox([VBox([Label('Cases'), q_cases]),
            VBox([Label('Deaths'), q_deaths])], layout=centered)])

In [21]:
S19.getleaderboard(ScoreboardCases, 1)

AttributeError: module 'Scoreboard19' has no attribute 'plotTD'

In [ ]:
S19.getleaderboard(ScoreboardCases, 4)

## Cumulative Death Counts

In [ ]:
S19.getleaderboard(ScoreboardDeaths, 1)

In [ ]:
S19.getleaderboard(ScoreboardDeaths 4)

### ADD TO GITHUB

In [ ]:
os.chdir(gitdir)
# #!git remote set-url origin https://github.com/ONYLAB/Scoreboard.git
# #!git config credential.helper store
#!git stash
#!git pull --rebase origin master
!git add --all
!git commit -am “Update”
!git push -u origin master
os.chdir(mypwd[0])